In [114]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 1000, 'display.max_colwidth', 1000, 'display.max_rows',1000)

# Data Read "elon-musk-tweets" file

In [124]:
df1 = pd.read_csv('data/elon-musk-tweets.csv')
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13610 entries, 0 to 13609
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   text            13590 non-null  object
 1   username        13610 non-null  object
 2   linktotweet     13610 non-null  object
 3   tweetembedcode  13610 non-null  object
 4   createdat       13610 non-null  object
dtypes: object(5)
memory usage: 531.8+ KB


In [125]:
start = '2020-03-15'
end = '2022-03-14'

In [126]:
df1[(df1.createdat >= start) & (df1.createdat <= end)][['text','createdat']]\
    .sort_values(by='createdat', ascending=False)\
    .reset_index(drop = True)

,text,createdat
0,"@haltman Little do they know, birds aren't real! https://t.co/mBPzUQMxRN",2022-03-11T19:41:00
1,@waitbutwhy What if one atom at a time in your body was switched out for another atom? There would be you with all new atoms and another you with the original atoms - Human of Theseus.,2022-03-11T00:31:00
2,@WholeMarsBlog @Erdayastronaut @CopSub This weekend,2022-03-10T05:44:00
3,"@FedorovMykhailo @OMarkarova You're welcome. We have also sent power adapters for car cigarette lighters, solar/battery packs and generators for places where electricity is not available.",2022-03-09T21:49:00
4,"@PPathole @SpaceX Optimized, fully-reusable Starship is ~150t to same reference orbit as Saturn V. In expendable mode, Starship payload would be 250t to 300t.",2022-03-09T21:41:00
...,...,...
6867,@engineers_feed,2020-03-16T20:13:00
6868,RT @Tesla: Model Y deliveries begin! https://t.co/ZhuiM5MTOf https://t.co/3gX6MBPmhp,2020-03-16T17:53:00
6869,"@Erdayastronaut @SpaceX @BoeingSpace @ulalaunch @Boeing Slight booster length increase to 70m, so 120m for whole system. Liftoff mass ~5000 mT.",2020-03-16T17:04:00
6870,Fear is the mind-killer,2020-03-15T03:33:00


# Data Read "Elon Musk 2" file

In [129]:
df2 = pd.read_csv('data/Elon Musk 2.csv')
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7918 entries, 0 to 7917
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   PostID         7918 non-null   int64 
 1   Time           7918 non-null   object
 2   TweetUrl       7918 non-null   object
 3   Content        7918 non-null   object
 4   UserID         7918 non-null   int64 
 5   UserName       7918 non-null   object
 6   RetweetNum     7918 non-null   int64 
 7   LikeNum        7918 non-null   int64 
 8   ReplyNum       7918 non-null   int64 
 9   UserHandle     7918 non-null   object
 10  UserUrl        7918 non-null   object
 11  Location       112 non-null    object
 12  UserID_PostID  7918 non-null   object
 13  CurrentPage    7918 non-null   int64 
dtypes: int64(6), object(8)
memory usage: 866.2+ KB


In [130]:
from datetime import datetime
months_in_year = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
months_in_year.index('Jan')

df2 = df2[['Content','Time']].sort_values(by='Time')

def time_convert(x):
    x = str.split(x)
    year = int(x[-1])
    mon = int(months_in_year.index(x[1])+1)
    day = int(x[2])
    time = x[3].split(':')
    hr = int(time[0])
    min = int(time[1])
    sec = int(time[2])
    date = datetime(year=year,month=mon,day=day,hour=hr, minute=min, second=sec)
    return date

df2['Time'] = df2.Time.apply(lambda x: time_convert(x))

df2[(df2.Time >= start) & (df2.Time <= end)].sort_values(by='Time',ascending=False)


,Content,Time
511,"@FedorovMykhailo @OMarkarova You’re welcome. We have also sent power adapters for car cigarette lighters, solar/battery packs and generators for places where electricity is not available.",2022-03-09 21:49:49
520,"@PPathole @SpaceX Optimized, fully-reusable Starship is ~150t to same reference orbit as Saturn V. In expendable mode, Starship payload would be 250t to 300t.",2022-03-09 21:41:57
513,"@Erdayastronaut @CopSub In our experience, COPV gain/pain flattens out above 6000 psi and is &lt;1 above 10,000 psi, unless volume constrained like Dragon. \n\nHeating from pressurization &amp; cooling from depressurization also become major problems at very high pressure.",2022-03-09 21:36:46
514,"@Kristennetten @BBCScienceNews There is no direct evidence of that ad being run, but that does sum up the sales pitch. It is mind-blowing that he &amp; his crew *rowed* to a whaling station from the Antarctic!",2022-03-09 19:29:56
525,@EvaFoxU Very important that there be an organization tasked with regulation removal or the number of rules will grow every year until progress is completely buried by bureaucracy!,2022-03-09 19:27:00
...,...,...
7142,Maybe worth considering chloroquine for C19 https://t.co/LEYob7Jofr,2020-03-16 20:31:48
7143,@engineers_feed 👍,2020-03-16 20:13:02
7146,"@Erdayastronaut @SpaceX @BoeingSpace @ulalaunch @Boeing Slight booster length increase to 70m, so 120m for whole system. Liftoff mass ~5000 mT.",2020-03-16 17:04:06
7137,Fear is the mind-killer,2020-03-15 03:33:30


## Tweet Content Cleaning

In [ ]:
def sentence_to_wordlist(sentence, remove_stopwords=False):
    # 1. Remove non-letters
    sentence_text = re.sub(r'[^\w\s]','', sentence)
    # 2. Remove all numbers
    sentence_text = re.sub(r'[0-9]+', '', sentence_text)
    # 3. Convert words to lower case and split them
    words = sentence_text.lower().split()
    # 4. remove website address   
    sentence = re.sub(r'^https?:\/\/.*[\r\n]*', '', sentence, flags=re.DOTALL)
    #sentence_text = re.sub(r'^https?:\/\/.*[\r\n]*', '', flags=re.MULTILINE)
    #6. drop the https
    #sentence = re.sub(r'[%https]', '', sentence)
    # 4. drop http
    p1=re.compile(r'http\S+', re.DOTALL)
    sentence = re.sub(p1, '', sentence)
    # 5. drop https
    p1=re.compile(r'https\S+', re.DOTALL)
    sentence = re.sub(p1, '', sentence)
    # 5. Stemming
    words = [stemmer.stem(w) for w in words] 
    # 6. Lemmatizing
    words = [lemmatizer.lemmatize(word) for word in words]
    # 7. Return a list of words
    return(words)